In [1]:
import tabula, camelot, os, time
import os, time
import pandas as pd
import numpy as np
import warnings
import requests
# warnings.simplefilter("ignore", UserWarning)
%cd ..

a:\Phong\Crawl_buffett


# get table from pdf

In [2]:
def convert_table(df):
    df.loc[-1] = df.columns
    df = df.sort_index()
    df.columns = np.arange(len(df.columns))
    return df.reset_index(drop=True)

def find_row_data(df, text):
    list_id = np.where((df[0].str.find(text) > 0) == True)[0]
    if len(list_id) == 0:
        return None
    else:
        return list_id[0] 
    
def get_vol_table_tabula(file_path, 
                         key_word = "期末発行済株式数(自己株式を含む)", 
                         type_get = 'volume'):
    if 'https' in file_path:
        tables = tabula.read_pdf(file_path, pages="all", multiple_tables=True, silent=True, stream=True)
    else:
        tables = tabula.read_pdf(file_path, pages="all", multiple_tables=True, silent=True)
    for table in range(len(tables)):
        df = pd.DataFrame(tables[table])
        if type_get == 'volume':
            df = convert_table(df)
            row_id = find_row_data(df, key_word)
            if row_id is not None and len(df.columns) < 10 and len(df[0][0]) < 100:
                return df
        else:
            print(df)
    return None


In [55]:
path_file = 'tests/Data/1301/PDF/2022_Q4_決算短信(2023_5_12).pdf'
tables = tabula.read_pdf(path_file, pages="all", multiple_tables=True, silent=True)
# tables = camelot.read_pdf(path_file, pages="all", multiple_tables=True, 
#                           flavor = 'stream', edge_tol=500,
#                           suppress_stdout=True, strip_text='\n')
# tables[1].df

In [52]:
df = pd.DataFrame(tables[4])
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,年間配当金,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,配当金総額,配当性向,純資産
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,配当率
1,NaN,第1四半期末,第2四半期末,第3四半期末,期末,NaN,合計,NaN,(合計),(連結),(連結)
2,NaN,円 銭,円 銭,円 銭,円 銭,NaN,円 銭,NaN,百万円,%,%
3,2022年3月期,―,―,―,90.00,NaN,90.00,NaN,972,20.9,2.3
4,2023年3月期,―,―,―,100.00,NaN,100.00,NaN,"1,076",18.5,2.4
5,2024年3月期(予想),―,―,―,90.00,NaN,90.00,NaN,NaN,16.1,NaN


In [53]:
def get_dividend_table(tables, 
                       year = 'Any', 
                       table_id = 4):
    df = pd.DataFrame(tables[table_id])
    df.dropna(subset=df.columns[0], inplace=True)
    df.dropna(axis=1, how='all', inplace=True)
    df.dropna(axis=1, thresh=3, inplace=True)
    df.columns = np.arange(len(df.columns))
    df.drop(columns=df.columns[5:], inplace=True)
    df.dropna(axis=0, how='all', inplace=True)
    df = df.reset_index(drop=True)
    df = df.iloc[[1]]
    df.columns = [year, 'Q1', 'Q2', 'Q3', 'Q4']
    df[year].iloc[0] = year
    if (pd.isna(df.iloc[0]) == False).all() and (df.iloc[0].str.len() < 10).all():
        return df.reset_index(drop=True)
    return pd.DataFrame(columns=['Year', 'Q1', 'Q2', 'Q3', 'Q4'])

get_dividend_table(tables, '2021')

,2021,Q1,Q2,Q3,Q4
0,2021,―,―,―,100.00


In [33]:
(df.iloc[0].str.len() < 10).all()

False

In [3]:
def cut_vol_table(tables):
    for i in range(len(tables)):
        df = tables[i].df
        df_find_key = df[0].str.find('期末発行済株式数（自己株式を含む）')
        for i in df_find_key.index:
            if df_find_key[i] >= 0:
                id_have_key = i
                df = df.iloc[id_have_key:, :].reset_index(drop=True)
                return df

def convert_table_mix_data(df):
    df_add = df[3].str.split(' ', expand=True)
    df_add2 = df[4].str.split(' ', expand=True)
    for i in df.index:
        if df[2][i] == '':
            df[2][i] = df_add[0][i]
            df[3][i] = df_add[1][i]
        if df[5][i] == '':
            df[5][i] = df_add2[1][i]
            df[4][i] = df_add2[0][i]
    return df
     
def drop_empty_col(df):
    for col in df.columns:
        if (df[col]=='').all():
            df = df.drop(columns=col)
    df.columns = np.arange(len(df.columns))
    return df

def get_vol_table_camelot(file_path):
    tables = camelot.read_pdf(file_path, pages="all", multiple_tables=True, flavor="stream", suppress_stdout=True)
    df = cut_vol_table(tables)
    # print(df)
    df = convert_table_mix_data(df)
    df = drop_empty_col(df)
    return df

In [4]:
def get_vol_table(file_path):
    df = get_vol_table_tabula(file_path)
    if df is None:
        df = get_vol_table_camelot(file_path)
    return df

In [ ]:
get_vol_table('tests/ocr.pdf')

In [7]:
# from volume.ocr_volume import ocr_pdf
from volume.get_volume import convert_pdf_to_text, find_row
# ocr_pdf('tests/Data/1301/PDF/2016_Q2_決算短信(2016_11_4).pdf')
text = convert_pdf_to_text('tests/ocr.pdf')
lst_data_of_time = find_row(text.replace(' ', '').replace('.', ','))
lst_data_of_time

[277210277, 942669]

# Get pdf

In [6]:
import setup
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
import time



class FinancailStatement(setup.Setup):
    def __init__(self):
        super().__init__('Selenium',source="VS")
    
    def get_data(self, link):
        self.driver.get(link)
        # time.sleep(5)
        soup = BeautifulSoup(self.driver.page_source,'html.parser',from_encoding='utf-8')
        return soup
    
    def get_table(self, id_company = 5486):
        print(f"https://www.buffett-code.com/company/{id_company}/library")
        soup = self.get_data(f"https://www.buffett-code.com/company/{id_company}/library")
        table = soup.find_all('table')
        return table, soup
    
    def get_pdf_link(self,link_):
        self.driver.get(link_)
        time.sleep(1)
        soup = BeautifulSoup(self.driver.page_source,'html.parser',from_encoding='utf-8')
        arr = soup.find_all('a')
        for i in arr:
            if i["href"].find("pdf") != -1:
                return i["href"]
        return ""


In [8]:
F = FinancailStatement()
table = F.get_table()
table

https://www.buffett-code.com/company/5486/library


[]

In [6]:
def create_link_df(table):
    json_company = {}
    for id_year, tr_year in enumerate(table[0].find_all('tr')):
        json_company_quy = {}
        year = ''
        for id_quy, td_quy in enumerate(tr_year.find_all('td')):
            lst_text, lst_link = [], []
            for li in td_quy.find_all('li'):
                if '決算短信' in li.text:
                    lst_text.append(li.text)
                    lst_link.append(f"https://www.buffett-code.com{li.find('a')['href']}")
            if id_quy != 0 and id_quy != 5:
                json_company_quy[f'Time_Q{id_quy}'] = lst_text
                json_company_quy[f'Link_Q{id_quy}'] = lst_link
                json_company_quy[f'Link_pdf_Q{id_quy}'] = np.nan
            if td_quy['class'][0] == 'center':
                year = td_quy.text
        if year != '':
            json_company[year] = json_company_quy.copy()

    df = pd.DataFrame(json_company).T.reset_index(drop=False)
    return df.rename(columns={'index': 'Year'})


In [7]:
def make_folder(id_company):
    try:
        os.mkdir(f'{id_company}')
        os.mkdir(f'{id_company}/PDF')
        os.mkdir(f'{id_company}/docs')
        os.mkdir(f'{id_company}/docx')
        os.mkdir(f'{id_company}/volume')
    except:
        pass


In [8]:
def save_check_point(F, id_company):
    if not os.path.exists(f'{id_company}/docs/link.csv'):
        table = F.get_table(id_company=id_company)
        df = create_link_df(table)
        df.to_csv(f'{id_company}/docs/link.csv', index=False)
        df_check = df.copy()
        for quy in ['Q1', 'Q2', 'Q3', 'Q4']:
            df_check[f'download_{quy}'] = np.nan
        df_check.to_csv(f'{id_company}/docs/check.csv', index=False)
    else:
        df = pd.read_csv(f'{id_company}/docs/link.csv')
        for quy in ['Q1', 'Q2', 'Q3', 'Q4']:
            df[f'Time_{quy}'] = df[f'Time_{quy}'].apply(lambda x: eval(x))
            df[f'Link_{quy}'] = df[f'Link_{quy}'].apply(lambda x: eval(x))
    return df

In [9]:
def get_download_pdf(F, id_company, df):
    df_check = pd.read_csv(f'{id_company}/docs/check.csv')

    for quy in ['Q1', 'Q2', 'Q3', 'Q4']:
        for id in df.index:
            if pd.isna(df_check[f'download_{quy}'][id]):
                for id_link in range(len(df[f'Time_{quy}'][id])):
                    year_ = df[f'Year'][id]
                    link_preview = df[f'Link_{quy}'][id][id_link]
                    if not 'https://www.buffett-code.com/company' in link_preview:
                        msg = 'Nan'
                    else:
                        try:
                            link_pdf = F.get_pdf_link(link_preview)
                            name = df[f'Time_{quy}'][id][id_link].replace(' ', '').replace('/', '_')
                            response = requests.get(link_pdf)
                            with open(f'{id_company}/PDF/{year_}_{quy}_{name}.pdf', 'wb') as f:
                                f.write(response.content)
                            msg = 'OK'
                        except:
                            msg = None
                    print(f'{id_company} - {year_} - {quy} - {id_link} - {msg} - {link_preview}')
                    df_check[f'download_{quy}'][id] = msg
                    df_check.to_csv(f'{id_company}/docs/check.csv', index=False)
                    time.sleep(20)

# get_download_pdf(F, id_company, df)

In [11]:
def save_pdf(id_company):
    make_folder(id_company)
    F = FinancailStatement()
    df = save_check_point(F, id_company)
    get_download_pdf(F, id_company, df)

save_pdf(5486)

5486 - 2019 - Q4 - 0 - None - https://www.buffett-code.com/company/5486/library/8b27f9d335f9a580017c4a/preview
5486 - 2019 - Q4 - 1 - None - https://www.buffett-code.com/company/5486/library/c95ee4aa4b5b6a56366610/preview
5486 - 2018 - Q4 - 0 - None - https://www.buffett-code.com/company/5486/library/f19bbefaa22f0f949e4a3e/preview
5486 - 2017 - Q4 - 0 - None - https://www.buffett-code.com/company/5486/library/7feea3a7795c38520300ed/preview
5486 - 2016 - Q4 - 0 - None - https://www.buffett-code.com/company/5486/library/dfdd1bf0d33d65a0344164/preview
5486 - 2015 - Q4 - 0 - None - https://www.buffett-code.com/company/5486/library/fe38a98df616afe1548a23/preview
5486 - 2014 - Q4 - 0 - None - https://www.buffett-code.com/company/5486/library/6e8e299b89421a19185548/preview
5486 - 2014 - Q4 - 1 - None - https://www.buffett-code.com/company/5486/library/0ada5cd2c2f35b928d8f5a/preview
5486 - 2013 - Q4 - 0 - None - https://www.buffett-code.com/company/5486/library/4951a05bc738bc116f46a0/preview
5